In [8]:
import pandas as pd
import numpy as np

from itertools import product

In [ ]:
data_skin = np.array(dataset['Pale_Skin'])
data_skin=np.maximum(data_skin, 0)
print(data_skin)
data_ = data_skin[np.where(data_skin==1)[0]]
print(data_.shape)

In [9]:


dataset = pd.read_csv("list_attr_celeba.txt", sep = ' ', header=1, skiprows = 0)

data = np.vstack((dataset['Male'], dataset["Young"], dataset["Mustache"], (-dataset["No_Beard"]),\
                                                                dataset["Bald"], dataset['Pale_Skin']))

# np.where(data > 0, data, 0)
data=np.maximum(data, 0)
print("0:Male, 1:Young, 2: Mustache, 3:Beard, 4:Bald")
print(data.sum(axis=1))
print(data)

0:Male, 1:Young, 2: Mustache, 3:Beard, 4:Bald
[ 84434 156734   8417  33441   4547   8701]
[[0 0 1 ... 1 0 0]
 [1 1 1 ... 1 1 1]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]]


In [21]:
data_0 = data[:-1,np.where(data[-1,:] == 0)[0]]
data_1 = data[:-1,np.where(data[-1,:] == 1)[0]]

In [35]:
print(data_0.shape)
print(data_1.shape)

(5, 193898)
(5, 8701)


In [26]:
import math
import os
import torch
import torch.distributions.constraints as constraints
import pyro
from pyro.optim import Adam
from pyro.infer import SVI, Trace_ELBO
import pyro.distributions as dist

## Training (data = data_0 or data = data_1)

In [78]:
data = data_0
model_save_path = './pyro_params_bias_0'

In [79]:



n_steps = max(int(data.shape[1]/200), 50)



# clear the param store in case we're in a REPL
pyro.clear_param_store()


def model(data):
    # sample f from the beta prior
    laten_sex = pyro.param('latent_sex', torch.tensor(0.5),
                   constraint=constraints.unit_interval)
    
    laten_mus_1 = pyro.param("latent_female_mustache", torch.tensor(0.5),
                   constraint=constraints.unit_interval)
    laten_mus_2 = pyro.param("latent_male_mustache", torch.tensor(0.5),
                   constraint=constraints.unit_interval)
    laten_mus = [laten_mus_1,laten_mus_2]
    
    laten_beard_1 = pyro.param("latent_no_mustache_beard", torch.tensor(0.5),
                   constraint=constraints.unit_interval)
    laten_beard_2 = pyro.param("latent_mustache_beard", torch.tensor(0.5),
                   constraint=constraints.unit_interval)
    laten_beard = [laten_beard_1,laten_beard_2]

    
    laten_young = pyro.param('latent_young', torch.tensor(0.5),
                   constraint=constraints.unit_interval)
    
    laten_bald_1 = pyro.param("latent_female_old_bald", torch.tensor(0.5),
                   constraint=constraints.unit_interval)
    laten_bald_2 = pyro.param("latent_female_young_bald", torch.tensor(0.5),
                   constraint=constraints.unit_interval)
    laten_bald_3 = pyro.param("latent_male_old_bald", torch.tensor(0.5),
                   constraint=constraints.unit_interval)
    laten_bald_4 = pyro.param("latent_male_young_bald", torch.tensor(0.5),
                   constraint=constraints.unit_interval)
    laten_bald = [[laten_bald_1,laten_bald_2],[laten_bald_3, laten_bald_4]]
    
    
    # loop over the observed data
    for i in pyro.plate("data_loop", data.shape[1]):
        # observe datapoint i using the bernoulli likelihood
        sex = pyro.sample('sex_{}'.format(i), dist.Bernoulli(laten_sex),obs = data[0,i])        
        young = pyro.sample('young_{}'.format(i), dist.Bernoulli(laten_young),obs = data[1,i])        

        mus = pyro.sample("mustache_{}".format(i), dist.Bernoulli(laten_mus[sex.long()]), obs=data[2,i])
        beard = pyro.sample("beard_{}".format(i), dist.Bernoulli(laten_beard[mus.long()]), obs=data[3,i])
        
        
        bald = pyro.sample("bald_{}".format(i), dist.Bernoulli(laten_bald[sex.long()][young.long()]), \
                                                             obs=data[4,i])
        


def guide(data):
    pass

store_p = []                       
# setup the optimizer
adam_params = {"lr": 0.1, "betas": (0.80, 0.999)}
optimizer = Adam(adam_params)
pyro.clear_param_store()
# setup the inference algorithm
svi = SVI(model, guide, optimizer, loss=Trace_ELBO())

# do gradient steps
for step in range(n_steps):
    rand_list = np.random.randint(low = 0, high=data.shape[1], size=1000)
#     svi.step(data[:,rand_list.tolist()])
    svi.step(torch.tensor(data[:,rand_list.tolist()]).float())
    if(step == 50):
        adam_params = {"lr": 0.01, "betas": (0.80, 0.999)}
        optimizer = Adam(adam_params)
    if(step == 100):
        adam_params = {"lr": 0.0005, "betas": (0.80, 0.999)}
        optimizer = Adam(adam_params)
    if(step %1 ==0):
        print('step: {}'.format(step))
#         for i in pyro.params.param_store.keys():
#             store_p[i] = pyro.params(i)
        p = pyro.get_param_store()
        store_p.append(p) 
        for (k,v) in p.items():
            print(k+": {:.3f}".format(v.item())) 




step: 0
latent_sex: 0.475
latent_female_mustache: 0.475
latent_male_mustache: 0.475
latent_no_mustache_beard: 0.475
latent_mustache_beard: 0.525
latent_young: 0.525
latent_female_old_bald: 0.475
latent_female_young_bald: 0.475
latent_male_old_bald: 0.475
latent_male_young_bald: 0.475
step: 1
latent_sex: 0.455
latent_female_mustache: 0.450
latent_male_mustache: 0.450
latent_no_mustache_beard: 0.450
latent_mustache_beard: 0.550
latent_young: 0.550
latent_female_old_bald: 0.451
latent_female_young_bald: 0.450
latent_male_old_bald: 0.450
latent_male_young_bald: 0.450
step: 2
latent_sex: 0.438
latent_female_mustache: 0.426
latent_male_mustache: 0.426
latent_no_mustache_beard: 0.426
latent_mustache_beard: 0.574
latent_young: 0.574
latent_female_old_bald: 0.426
latent_female_young_bald: 0.426
latent_male_old_bald: 0.426
latent_male_young_bald: 0.426
step: 3
latent_sex: 0.420
latent_female_mustache: 0.402
latent_male_mustache: 0.402
latent_no_mustache_beard: 0.402
latent_mustache_beard: 0.598


step: 29
latent_sex: 0.433
latent_female_mustache: 0.094
latent_male_mustache: 0.125
latent_no_mustache_beard: 0.138
latent_mustache_beard: 0.898
latent_young: 0.777
latent_female_old_bald: 0.097
latent_female_young_bald: 0.094
latent_male_old_bald: 0.132
latent_male_young_bald: 0.101
step: 30
latent_sex: 0.433
latent_female_mustache: 0.091
latent_male_mustache: 0.123
latent_no_mustache_beard: 0.137
latent_mustache_beard: 0.902
latent_young: 0.776
latent_female_old_bald: 0.093
latent_female_young_bald: 0.091
latent_male_old_bald: 0.130
latent_male_young_bald: 0.097
step: 31
latent_sex: 0.429
latent_female_mustache: 0.087
latent_male_mustache: 0.121
latent_no_mustache_beard: 0.135
latent_mustache_beard: 0.906
latent_young: 0.775
latent_female_old_bald: 0.089
latent_female_young_bald: 0.087
latent_male_old_bald: 0.129
latent_male_young_bald: 0.094
step: 32
latent_sex: 0.422
latent_female_mustache: 0.084
latent_male_mustache: 0.118
latent_no_mustache_beard: 0.134
latent_mustache_beard: 0.

step: 58
latent_sex: 0.428
latent_female_mustache: 0.040
latent_male_mustache: 0.104
latent_no_mustache_beard: 0.133
latent_mustache_beard: 0.951
latent_young: 0.766
latent_female_old_bald: 0.041
latent_female_young_bald: 0.040
latent_male_old_bald: 0.111
latent_male_young_bald: 0.048
step: 59
latent_sex: 0.434
latent_female_mustache: 0.039
latent_male_mustache: 0.103
latent_no_mustache_beard: 0.134
latent_mustache_beard: 0.952
latent_young: 0.765
latent_female_old_bald: 0.040
latent_female_young_bald: 0.039
latent_male_old_bald: 0.111
latent_male_young_bald: 0.047
step: 60
latent_sex: 0.433
latent_female_mustache: 0.038
latent_male_mustache: 0.103
latent_no_mustache_beard: 0.134
latent_mustache_beard: 0.952
latent_young: 0.766
latent_female_old_bald: 0.039
latent_female_young_bald: 0.038
latent_male_old_bald: 0.111
latent_male_young_bald: 0.046
step: 61
latent_sex: 0.432
latent_female_mustache: 0.038
latent_male_mustache: 0.103
latent_no_mustache_beard: 0.135
latent_mustache_beard: 0.

step: 87
latent_sex: 0.408
latent_female_mustache: 0.023
latent_male_mustache: 0.099
latent_no_mustache_beard: 0.134
latent_mustache_beard: 0.964
latent_young: 0.770
latent_female_old_bald: 0.025
latent_female_young_bald: 0.023
latent_male_old_bald: 0.110
latent_male_young_bald: 0.033
step: 88
latent_sex: 0.405
latent_female_mustache: 0.023
latent_male_mustache: 0.100
latent_no_mustache_beard: 0.133
latent_mustache_beard: 0.964
latent_young: 0.771
latent_female_old_bald: 0.024
latent_female_young_bald: 0.023
latent_male_old_bald: 0.109
latent_male_young_bald: 0.032
step: 89
latent_sex: 0.407
latent_female_mustache: 0.023
latent_male_mustache: 0.100
latent_no_mustache_beard: 0.133
latent_mustache_beard: 0.965
latent_young: 0.771
latent_female_old_bald: 0.024
latent_female_young_bald: 0.023
latent_male_old_bald: 0.107
latent_male_young_bald: 0.032
step: 90
latent_sex: 0.407
latent_female_mustache: 0.022
latent_male_mustache: 0.100
latent_no_mustache_beard: 0.133
latent_mustache_beard: 0.

step: 116
latent_sex: 0.425
latent_female_mustache: 0.016
latent_male_mustache: 0.104
latent_no_mustache_beard: 0.136
latent_mustache_beard: 0.969
latent_young: 0.764
latent_female_old_bald: 0.016
latent_female_young_bald: 0.016
latent_male_old_bald: 0.113
latent_male_young_bald: 0.026
step: 117
latent_sex: 0.425
latent_female_mustache: 0.016
latent_male_mustache: 0.104
latent_no_mustache_beard: 0.135
latent_mustache_beard: 0.969
latent_young: 0.764
latent_female_old_bald: 0.016
latent_female_young_bald: 0.016
latent_male_old_bald: 0.113
latent_male_young_bald: 0.026
step: 118
latent_sex: 0.420
latent_female_mustache: 0.015
latent_male_mustache: 0.104
latent_no_mustache_beard: 0.135
latent_mustache_beard: 0.969
latent_young: 0.763
latent_female_old_bald: 0.016
latent_female_young_bald: 0.016
latent_male_old_bald: 0.113
latent_male_young_bald: 0.026
step: 119
latent_sex: 0.412
latent_female_mustache: 0.015
latent_male_mustache: 0.104
latent_no_mustache_beard: 0.134
latent_mustache_beard

...

step: 841
latent_sex: 0.425
latent_female_mustache: 0.001
latent_male_mustache: 0.096
latent_no_mustache_beard: 0.137
latent_mustache_beard: 0.972
latent_young: 0.767
latent_female_old_bald: 0.001
latent_female_young_bald: 0.001
latent_male_old_bald: 0.119
latent_male_young_bald: 0.020
step: 842
latent_sex: 0.428
latent_female_mustache: 0.001
latent_male_mustache: 0.096
latent_no_mustache_beard: 0.136
latent_mustache_beard: 0.972
latent_young: 0.766
latent_female_old_bald: 0.001
latent_female_young_bald: 0.001
latent_male_old_bald: 0.116
latent_male_young_bald: 0.020
step: 843
latent_sex: 0.438
latent_female_mustache: 0.001
latent_male_mustache: 0.096
latent_no_mustache_beard: 0.135
latent_mustache_beard: 0.972
latent_young: 0.761
latent_female_old_bald: 0.001
latent_female_young_bald: 0.001
latent_male_old_bald: 0.117
latent_male_young_bald: 0.020
step: 844
latent_sex: 0.442
latent_female_mustache: 0.001
latent_male_mustache: 0.096
latent_no_mustache_beard: 0.135
latent_mustache_beard

step: 870
latent_sex: 0.430
latent_female_mustache: 0.001
latent_male_mustache: 0.101
latent_no_mustache_beard: 0.138
latent_mustache_beard: 0.971
latent_young: 0.768
latent_female_old_bald: 0.001
latent_female_young_bald: 0.001
latent_male_old_bald: 0.112
latent_male_young_bald: 0.020
step: 871
latent_sex: 0.426
latent_female_mustache: 0.001
latent_male_mustache: 0.103
latent_no_mustache_beard: 0.136
latent_mustache_beard: 0.972
latent_young: 0.768
latent_female_old_bald: 0.001
latent_female_young_bald: 0.001
latent_male_old_bald: 0.115
latent_male_young_bald: 0.020
step: 872
latent_sex: 0.425
latent_female_mustache: 0.001
latent_male_mustache: 0.104
latent_no_mustache_beard: 0.134
latent_mustache_beard: 0.973
latent_young: 0.767
latent_female_old_bald: 0.001
latent_female_young_bald: 0.001
latent_male_old_bald: 0.117
latent_male_young_bald: 0.020
step: 873
latent_sex: 0.432
latent_female_mustache: 0.001
latent_male_mustache: 0.105
latent_no_mustache_beard: 0.133
latent_mustache_beard

step: 899
latent_sex: 0.405
latent_female_mustache: 0.001
latent_male_mustache: 0.105
latent_no_mustache_beard: 0.136
latent_mustache_beard: 0.973
latent_young: 0.781
latent_female_old_bald: 0.001
latent_female_young_bald: 0.001
latent_male_old_bald: 0.129
latent_male_young_bald: 0.022
step: 900
latent_sex: 0.410
latent_female_mustache: 0.001
latent_male_mustache: 0.104
latent_no_mustache_beard: 0.138
latent_mustache_beard: 0.973
latent_young: 0.785
latent_female_old_bald: 0.001
latent_female_young_bald: 0.001
latent_male_old_bald: 0.127
latent_male_young_bald: 0.021
step: 901
latent_sex: 0.422
latent_female_mustache: 0.001
latent_male_mustache: 0.102
latent_no_mustache_beard: 0.139
latent_mustache_beard: 0.973
latent_young: 0.786
latent_female_old_bald: 0.001
latent_female_young_bald: 0.001
latent_male_old_bald: 0.123
latent_male_young_bald: 0.021
step: 902
latent_sex: 0.434
latent_female_mustache: 0.001
latent_male_mustache: 0.102
latent_no_mustache_beard: 0.140
latent_mustache_beard

step: 928
latent_sex: 0.427
latent_female_mustache: 0.001
latent_male_mustache: 0.101
latent_no_mustache_beard: 0.132
latent_mustache_beard: 0.966
latent_young: 0.773
latent_female_old_bald: 0.001
latent_female_young_bald: 0.001
latent_male_old_bald: 0.120
latent_male_young_bald: 0.020
step: 929
latent_sex: 0.430
latent_female_mustache: 0.001
latent_male_mustache: 0.100
latent_no_mustache_beard: 0.129
latent_mustache_beard: 0.965
latent_young: 0.771
latent_female_old_bald: 0.001
latent_female_young_bald: 0.001
latent_male_old_bald: 0.122
latent_male_young_bald: 0.020
step: 930
latent_sex: 0.433
latent_female_mustache: 0.001
latent_male_mustache: 0.099
latent_no_mustache_beard: 0.129
latent_mustache_beard: 0.965
latent_young: 0.769
latent_female_old_bald: 0.001
latent_female_young_bald: 0.001
latent_male_old_bald: 0.125
latent_male_young_bald: 0.020
step: 931
latent_sex: 0.429
latent_female_mustache: 0.001
latent_male_mustache: 0.096
latent_no_mustache_beard: 0.129
latent_mustache_beard

step: 957
latent_sex: 0.412
latent_female_mustache: 0.001
latent_male_mustache: 0.102
latent_no_mustache_beard: 0.127
latent_mustache_beard: 0.970
latent_young: 0.769
latent_female_old_bald: 0.001
latent_female_young_bald: 0.001
latent_male_old_bald: 0.114
latent_male_young_bald: 0.019
step: 958
latent_sex: 0.409
latent_female_mustache: 0.001
latent_male_mustache: 0.100
latent_no_mustache_beard: 0.126
latent_mustache_beard: 0.970
latent_young: 0.770
latent_female_old_bald: 0.001
latent_female_young_bald: 0.001
latent_male_old_bald: 0.114
latent_male_young_bald: 0.019
step: 959
latent_sex: 0.411
latent_female_mustache: 0.001
latent_male_mustache: 0.100
latent_no_mustache_beard: 0.126
latent_mustache_beard: 0.970
latent_young: 0.769
latent_female_old_bald: 0.001
latent_female_young_bald: 0.001
latent_male_old_bald: 0.115
latent_male_young_bald: 0.019
step: 960
latent_sex: 0.420
latent_female_mustache: 0.001
latent_male_mustache: 0.098
latent_no_mustache_beard: 0.126
latent_mustache_beard

In [80]:

all_param = pyro.get_param_store()
print(all_param)
all_param.save(model_save_path)

In [81]:
graph_params = pyro.params.param_store.ParamStoreDict()
graph_params.load(model_save_path)

from pyro import poutine
from pyro.infer import config_enumerate, infer_discrete

def determined_model():
    laten_sex = graph_params.get_param('latent_sex')

    laten_mus_1 = graph_params.get_param("latent_female_mustache")
    laten_mus_2 = graph_params.get_param("latent_male_mustache")
    laten_mus = [laten_mus_1, laten_mus_2]

    laten_beard_1 = graph_params.get_param("latent_no_mustache_beard")
    laten_beard_2 = graph_params.get_param("latent_mustache_beard")
    laten_beard = [laten_beard_1, laten_beard_2]

    laten_young = graph_params.get_param('latent_young')

    laten_bald_1 = graph_params.get_param("latent_female_old_bald")
    laten_bald_2 = graph_params.get_param("latent_female_young_bald")
    laten_bald_3 = graph_params.get_param("latent_male_old_bald")
    laten_bald_4 = graph_params.get_param("latent_male_young_bald")
    laten_bald = [[laten_bald_1, laten_bald_2], [laten_bald_3, laten_bald_4]]

    with pyro.plate("a_plate", size=1, dim=-2):
        sex = pyro.sample('sex', dist.Bernoulli(laten_sex))
        young = pyro.sample('young', dist.Bernoulli(laten_young))
        #         with pyro.plate("b_plate", size=1, dim=-1):

        mus = pyro.sample("mustache", dist.Bernoulli(laten_mus[sex.long()]))
        beard = pyro.sample("beard", dist.Bernoulli(laten_beard[mus.long()]))

        bald = pyro.sample("bald", dist.Bernoulli(laten_bald[sex.long()][young.long()]))

        
def make_log_joint(model):

    def _log_joint(data, *args, **kwargs):

        conditioned_model = poutine.condition(model, data=data)

        trace = poutine.trace(conditioned_model).get_trace(*args, **kwargs)

        return trace.log_prob_sum()

    return _log_joint

 

scale_log_joint = make_log_joint(determined_model)

# print(scale_log_joint({\
#                        "sex": torch.tensor(0.0),
#                        "young": torch.tensor(0.),
#                        "mustache": torch.tensor(0.0),
#                        "beard": torch.tensor(.0),
#                        "bald":torch.tensor(.0)\
# }).exp())
from itertools import product

perm_array = np.array(list(product([0, 1], repeat=5))).astype(float)

def joint_prob_graph(arr):
    prob = scale_log_joint({\
                       "sex": torch.tensor(arr[0]),
                       "young": torch.tensor(arr[1]),
                       "mustache": torch.tensor(arr[2]),
                       "beard": torch.tensor(arr[3]),
                       "bald":torch.tensor(arr[4])\
                           }).exp()
    return prob.data.numpy()

learned_joint_prob_array = []

for arr in perm_array:
    p = joint_prob_graph(arr)
    learned_joint_prob_array.append(p)

learned_joint_prob_array = np.array(learned_joint_prob_array)


ori_joint_prob_array = []

for arr in perm_array:
    p = ((data.T == arr).sum(axis=1) == 5).mean()
    ori_joint_prob_array.append(p)

ori_joint_prob_array = np.array(ori_joint_prob_array)

print(np.mean((ori_joint_prob_array - learned_joint_prob_array)**2)**0.5)

0.03212564423971267
